In [4]:
from slt_positional_bias.dataset import generate_data_frame, generate_merged_data_frame

df_merged = generate_merged_data_frame()
df_merged


2025-07-17 18:45:30.475 | INFO     | slt_positional_bias.config:<module>:11 - PROJ_ROOT path is: C:\Users\marti\slt_group_2_positional_bias


2025-07-17 18:45:30.780 | INFO     | slt_positional_bias.dataset:generate_data_frame:48 - file path exists
2025-07-17 18:45:37.091 | INFO     | slt_positional_bias.dataset:generate_data_frame:48 - file path exists
2025-07-17 18:45:37.096 | INFO     | slt_positional_bias.dataset:generate_data_frame:48 - file path exists


,qid,doc_id,rel,text_x,text_y
0,2024-145979,msmarco_v2.1_doc_01_523681915#0_449763684,2,what is vicarious trauma and how can it be cop...,Vicarious traumatization - Wikipedia Vicarious...
1,2024-145979,msmarco_v2.1_doc_13_1647729865#0_3617397938,3,what is vicarious trauma and how can it be cop...,Vicarious Trauma: What Causes it and Ways to H...
2,2024-145979,msmarco_v2.1_doc_25_1148700328#0_2179512504,2,what is vicarious trauma and how can it be cop...,What is Vicarious Trauma? | What is Vicarious ...
3,2024-145979,msmarco_v2.1_doc_25_1148700328#1_2179514079,2,what is vicarious trauma and how can it be cop...,What is Vicarious Trauma? | What is Vicarious ...
4,2024-145979,msmarco_v2.1_doc_44_1043805224#2_2182641075,2,what is vicarious trauma and how can it be cop...,What is Vicarious Trauma? - Jefferson Center -...
...,...,...,...,...,...
108473,2024-21669,msmarco_v2.1_doc_08_298503251#1_568833050,0,NaN,NaN
108474,2024-21669,msmarco_v2.1_doc_27_877180220#2_1765802089,1,NaN,NaN
108475,2024-21669,msmarco_v2.1_doc_01_1752638978#7_2584141250,1,NaN,NaN
108476,2024-21669,msmarco_v2.1_doc_09_914336366#11_1554208777,0,NaN,NaN


In [16]:
print(df_merged['text_y'].dropna().iloc[0])

Vicarious traumatization - Wikipedia Vicarious traumatization
Vicarious traumatization
Contents
Signs and symptoms
Contributing factors
Related concepts
Mechanism
Measurement
Addressing
See also
References
Further reading
External links
 Vicarious traumatization - Wikipedia
Vicarious traumatization
From Wikipedia, the free encyclopedia
Jump to navigation Jump to search

This article's text uses more words than are necessary. Please help improve this article by using fewer words whilst keeping the content of the article. ( January 2018)
Vicarious traumatization ( VT) is a transformation in the self of a trauma worker or helper that results from empathic engagement with traumatized clients and their reports of traumatic experiences. It is a special form of countertransference stimulated by exposure to the client's traumatic material. Its hallmark is disrupted spirituality, or a disruption in the trauma workers' perceived meaning and hope. The term was coined specifically with reference t

In [ ]:
API_URL = "https://api.helmholtz-blablador.fz-juelich.de/v1/"
API_KEY = ""
API_MODEL = "alias-fast"

from openai import OpenAI

client = OpenAI(
    api_key=API_KEY,
    base_url=API_URL
)

In [90]:
import pandas as pd

df_results_all = pd.DataFrame()

In [91]:

positions = [0, 2, 4, 7, 9]  # Positionen für das relevante Dokument
rel0_count = 9               # Anzahl irrelevanter Dokumente pro Sample
sample_count = rel0_count + 1 # Anzahl Dokumente im Prompt (inkl. rel=3)
max_qids = 25                # Maximale Anzahl qids, die verarbeitet werden

In [93]:

import pandas as pd

# Liste für Ergebnisse
results = []

# Filter gültige qids (mit rel=3 und keinem NaN in text_x/text_y)
valid_qids = []
for qid in df_merged['qid'].unique():
    df_qid = df_merged[df_merged['qid'] == qid]
    # Check ob es mindestens eine Zeile mit non-NaN in text_x und text_y gibt
    if df_qid[['text_x', 'text_y']].notna().all(axis=1).any():
        if df_qid['rel'].isin([3]).any():
            rel3 = df_qid[df_qid['rel'] == 3].dropna(subset=['text_x', 'text_y'])
            rel0 = df_qid[df_qid['rel'] == 0].dropna(subset=['text_y'])
            if not rel3.empty and len(rel0) >= rel0_count:
                valid_qids.append(qid)
    if len(valid_qids) == max_qids:
        break

# Iteriere über gültige qids
for qid in valid_qids:
    df_qid = df_merged[df_merged['qid'] == qid]

    # Ein relevantes und k-1 irrelevante Dokumente
    rel_3_doc = df_qid[(df_qid['rel'] == 3)].dropna(subset=['text_x', 'text_y']).head(1)
    rel_0_docs = df_qid[(df_qid['rel'] == 0)].dropna(subset=['text_y']).head(rel0_count)

    # Extrahiere Texte
    rel_3_text = rel_3_doc['text_y'].iloc[0]
    query = rel_3_doc['text_x'].iloc[0]
    rel_0_texts = rel_0_docs['text_y'].tolist()

    for pos in positions:
        # Kontext erstellen
        text_list = rel_0_texts.copy()
        if pos >= len(text_list):
            text_list.append(rel_3_text)
        else:
            text_list.insert(pos, rel_3_text)
        context_string = ' '.join(text_list)

        # Prompt erstellen
        user_prompt = f"""
        Context:
        {context_string}

        Question:
        {query}

        Answer:"""
        system_prompt = "You are a helpful assistant answering a question based on retrieved context information."

        # Modell antworten lassen
        response = client.chat.completions.create(
            model=API_MODEL,
            temperature=0.7,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
        )
        answer = response.choices[0].message.content.strip()

        # Evaluation vorbereiten
        evaluation_prompt = f"""
        You are evaluating a language model's answer based on a context of {sample_count} sections: one highly relevant, and {rel0_count} irrelevant.

        Relevant section:
        \"\"\"{rel_3_text}\"\"\"

        Answer from the model:
        \"\"\"{answer}\"\"\"

        Does the answer contain information from the relevant section?

        Reply with exactly one word: Yes or No.
        """
        eval_response = client.chat.completions.create(
            model=API_MODEL,
            temperature=0,
            messages=[{"role": "user", "content": evaluation_prompt}],
        )
        eval_result = eval_response.choices[0].message.content.strip()
        print(eval_result)

        # Ergebnis speichern
        results.append({
            'qid': qid,
            'position': pos + 1, 
            'eval_result': eval_result
        })

# In DataFrame umwandeln
df_results = pd.DataFrame(results)

# Ergebnisse an den globalen Speicher anhängen
try:
    df_results_all = pd.concat([df_results_all, df_results], ignore_index=True)
except NameError:
    df_results_all = df_results.copy()

print("\nEval-Ergebnisse (Stichprobe):")
print(df_results_all['eval_result'].value_counts())


Yes
Yes
Yes
Yes
Yes
No
No
No
Yes
No
No
Yes
No
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
No
No
No
No
No
Yes
No
No
No
No
No
Yes
No
Yes
Yes
Yes
Yes
Yes
Yes
Yes
No
No
No
No
No
No
No
No
No
No
Yes
No
No
No
Yes
Yes
Yes
Yes
Yes
Yes
No
No
No
No
No
Yes
No
Yes
No
Yes
Yes
Yes
Yes
No
Yes
Yes
Yes
Yes
Yes
Yes
No
No
Yes
Yes
No
No
No
No
No
No
Yes
Yes
Yes
Yes
Yes
No
No
No
No
No
No.
No
No
No
No
Yes
No
Yes
Yes
Yes
Yes
No
No
No
No
No
No
No
No
No

Eval-Ergebnisse (Stichprobe):
eval_result
No     65
Yes    59
No.     1
Name: count, dtype: int64


In [94]:
df_results_all['is_yes'] = df_results_all['eval_result'].apply(
    lambda x: x.split("</think>")[-1].strip().lower().startswith("yes")
).astype(int)

u_curve_df = df_results_all.groupby('position')['is_yes'].agg(['mean', 'count'])
u_curve_df.rename(columns={'mean': 'yes_rate', 'count': 'num_samples'}, inplace=True)
u_curve_df['yes_rate_pct'] = (u_curve_df['yes_rate'] * 100).round(1).astype(str) + '%'

print("\nKumulierte U-Kurve:")
print(u_curve_df[['yes_rate_pct', 'num_samples']])


Kumulierte U-Kurve:
         yes_rate_pct  num_samples
position                          
1               52.0%           25
3               40.0%           25
5               44.0%           25
8               48.0%           25
10              52.0%           25
